# Cube Semantic Layer

SQL Chains and Agents support building and running SQL queries based on natural language prompts. 
However, when faced with complex queries, they may not perform well in producing results.
Now, I'm providing a Cube Semantic Layer Agent, which can better support complex queries. 
A Cube data model is used to transform raw data into meaningful business definitions and pre-aggregate data for optimal results.

They enable use cases such as:

- Generating queries that will be run based on natural language questions
- Creating chatbots that can answer questions based on database data
- Building custom dashboards based on insights a user wants to analyze

## Overview

LangChain provides tools to interact with Cube:

1. `Build Cube queries` based on natural language user questions
2. `Query a Cube` using chains for query creation and execution
3. `Interact with a Cube` using agents for robust and flexible querying 

![cube_semantic_layer_agent.png](/img/cube_semantic_layer_agent.png)

## Quickstart

First, get required packages and set environment variables:

In [2]:
! pip install langchain openai

# Set env var OPENAI_API_KEY or load from a .env file
# import dotenv

# dotenv.load_dotenv()

Sample data for this notebook is sourced from the Cube examples, available at [active-users-example](https://github.com/cube-js/cube/tree/master/examples/recipes/active-users).
 
* Clone the repo [https://github.com/cube-js/cube](https://github.com/cube-js/cube)
* Go to the `examples/recipes/active-users` directory
* Run `docker-compose up`

Let's create a `create_cube_agent` to create and execute Cube queries.

In [3]:
from pydantic.v1 import SecretStr

from langchain.agents import create_cube_agent
from langchain.agents.agent_toolkits import CubeToolkit
from langchain.chat_models import ChatOpenAI
from langchain.utilities.cube import Cube

# create Cube client
cube = Cube(
    cube_api_url="http://localhost:4000/cubejs-api/v1",
    cube_api_token=SecretStr("CUBE_API_TOKEN"),
)

# create Cube Toolkit
toolkit = CubeToolkit(
    cube=cube,
    llm=ChatOpenAI(temperature=0, verbose=True),
)

# create cube agent
agent_executor = create_cube_agent(
    llm=ChatOpenAI(verbose=True),
    toolkit=toolkit,
    verbose=True,
)

In [4]:
agent_executor.run(
    "I'd like to get the data on our daily, weekly, and monthly active users."
)



> Entering new AgentExecutor chain...
Action: list_models_cube
Action Input: ""
Observation: | Model | Description |
| --- | --- |
| ActiveUsers | Active Users |

Thought:The ActiveUsers model seems relevant. I should query the meta-information of this model to see what columns are available.
Action: meta_information_cube
Action Input: "ActiveUsers"
Observation: ## Model: ActiveUsers

### Measures:
| Title | Description | Column | Type |
| --- | --- | --- | --- |
| Monthly Active Users |  | ActiveUsers.monthlyActiveUsers | number |
| Weekly Active Users |  | ActiveUsers.weeklyActiveUsers | number |
| Daily Active Users |  | ActiveUsers.dailyActiveUsers | number |
| WAU to MAU |  | ActiveUsers.wauToMau | number |

### Dimensions:
| Title | Description | Column | Type |
| --- | --- | --- | --- |
| Created at |  | ActiveUsers.createdAt | time |

Thought:I can query the ActiveUsers model to get the data on daily, weekly, and monthly active users. I will include the measures "Daily Active

'The daily, weekly, and monthly active users are all 700.'

## Text-to-Cube query


In [3]:
from langchain.chains import create_cube_query_chain

Let's create the chain that will build the Cube Query:


In [4]:
chain = create_cube_query_chain(ChatOpenAI(temperature=0), cube)
response = chain.invoke({"question": "Get WAU to MAU active users."})
print(response.json(indent=2, exclude_unset=True))

{
  "measures": [
    "ActiveUsers.wauToMau"
  ],
  "limit": 5
}


After building the Cube query based on a user question, we can execute the load:

In [5]:
cube.load(response)

{'query': {'measures': ['ActiveUsers.wauToMau'],
  'limit': 5,
  'rowLimit': 5,
  'timezone': 'UTC',
  'order': [],
  'filters': [],
  'dimensions': [],
  'timeDimensions': []},
 'data': [{'ActiveUsers.wauToMau': '100.0000000000000000'}],
 'lastRefreshTime': '2023-10-25T15:55:51.204Z',
 'refreshKeyValues': [],
 'usedPreAggregations': {},
 'transformedQuery': {'sortedDimensions': [],
  'sortedTimeDimensions': [],
  'timeDimensions': [],
  'measures': ['ActiveUsers.wauToMau'],
  'leafMeasureAdditive': False,
  'leafMeasures': ['ActiveUsers.weeklyActiveUsers',
   'ActiveUsers.monthlyActiveUsers'],
  'hasNoTimeDimensionsWithoutGranularity': True,
  'allFiltersWithinSelectedDimensions': True,
  'isAdditive': False,
  'granularityHierarchies': {'year': ['year',
    'quarter',
    'month',
    'month',
    'day',
    'hour',
    'minute',
    'second'],
   'quarter': ['quarter', 'month', 'day', 'hour', 'minute', 'second'],
   'month': ['month', 'day', 'hour', 'minute', 'second'],
   'week': [

As we can see, the Cube Query Builder chain **only created** the query, and we handled the **query execution separately**.

Get the SQL Code generated by Cube to be executed in the database.

In [6]:
cube.sql(response)

{'external': False,
 'sql': ['SELECT 100.000 * "active_users__weekly_active_users" / NULLIF("active_users__monthly_active_users", 0) "active_users__wau_to_mau" FROM (SELECT count(distinct "active_users_weekly_active_users_cumulative___active_users".user_id) "active_users__weekly_active_users" FROM (SELECT user_id, created_at FROM public.orders) AS "active_users_weekly_active_users_cumulative___active_users" ) as q_0 , (SELECT count(distinct "active_users_monthly_active_users_cumulative___active_users".user_id) "active_users__monthly_active_users" FROM (SELECT user_id, created_at FROM public.orders) AS "active_users_monthly_active_users_cumulative___active_users" ) as q_1 LIMIT 5',
  []],
 'order': {},
 'cacheKeyQueries': [['SELECT FLOOR((UNIX_TIMESTAMP()) / 10) as refresh_key',
   [],
   {'external': True, 'renewalThreshold': 10}]],
 'preAggregations': [],
 'dataSource': 'default',
 'aliasNameToMember': {'active_users__wau_to_mau': 'ActiveUsers.wauToMau'},
 'rollupMatchResults': [],
 '